In [ ]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

In [1]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC

from sklearn.model_selection import StratifiedKFold 

model = LinearSVC(dual=False,tol=1e-3, class_weight = 'balanced') #please try with different versions of gamma and 'C' parameters here
                  
df = pd.read_csv('telekomexportsichter11.csv')
#df1 = pd.read_csv('telekomexportsichter12.csv')
#df2 = pd.read_csv('telekomexportsichter13.csv')
#df3 = pd.read_csv('telekomexportsichter14.csv')
#df4 = pd.read_csv('telekomexportsichter15.csv')
#df5 = pd.read_csv('telekomexportsichter16.csv')
#df6 = pd.read_csv('telekomexportsichter17.csv')
#df7 = pd.read_csv('telekomexportsichter18.csv')

#df = df.append(df1)
#df = df.append(df2)
#df = df.append(df3)
#df = df.append(df4)
#df = df.append(df5)
#df = df.append(df6)
#df = df.append(df7)


priorCases = df[(df['Tag'] == 'Prio-Fall') & (df['kind'] == 1)]['Content']
priorCases = pd.Series.to_frame(priorCases)

nonPriorCases = df[(df['Tag'] != 'Prio-Fall') & (df['kind'] == 1)]['Content']
nonPriorCases = pd.Series.to_frame(nonPriorCases)

priorCases = priorCases.assign(label = 1)
nonPriorCases = nonPriorCases.assign(label = 0)
#priorCases.drop(priorCases.index[priorCases['Content'].isnull()])
#priorCases[priorCases['Content'].isnull()].head
priorCases = priorCases.append(nonPriorCases)

In [2]:
priorCases = priorCases.drop(priorCases.index[priorCases['Content'].isnull()])
cv = CountVectorizer(min_df = 3, max_df = 10000, ngram_range=(1, 3), lowercase =True) #change the ngram range to 5,7, maximum 9 

In [111]:
print(priorCases.index[priorCases['Content'].isnull()])

Int64Index([], dtype='int64')


In [3]:
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=0) #do not use more or less splits
scores = []
for train_index, test_index in skf.split(priorCases['Content'] , priorCases['label']):
    #priorCases.loc[train_index]['Content']
    
    model = LinearSVC(dual=False,tol=1e-3, class_weight = 'balanced')
    priorCases2 = priorCases.loc[train_index]
    priorCases2 = priorCases2.drop(priorCases2.index[priorCases2['Content'].isnull()])
    X_train_counts = cv.fit_transform(priorCases2['Content'])
    #print(X_train_counts.shape)


    tfidfTransform = TfidfTransformer()
    X_Train_tfidf = tfidfTransform.fit_transform(X_train_counts)
    #print(X_Train_tfidf.shape)
    
    priorCases2 = priorCases.loc[test_index]
    priorCases2 = priorCases2.drop(priorCases2.index[priorCases2['Content'].isnull()])
    X_test_counts = cv.transform(priorCases2['Content'])
    #print(X_test_counts.shape)

    X_test_tfidf = tfidfTransform.transform(X_test_counts)
    #print(X_test_tfidf.shape)
    
    priorCases2 = priorCases.loc[train_index]
    priorCases2 = priorCases2.drop(priorCases2.index[priorCases2['Content'].isnull()])
    model.fit(X_Train_tfidf, priorCases2['label'])
    
    pred = model.predict(X_test_tfidf)
    
    priorCases2 = priorCases.loc[test_index]
    priorCases2 = priorCases2.drop(priorCases2.index[priorCases2['Content'].isnull()])
    answer = sklearn.metrics.f1_score(priorCases2['label'], pred)
    #print(sklearn.metrics.confusion_matrix(priorCases2['label'], pred, labels=None, sample_weight=None))
    print(np.mean(pred == priorCases2['label']))
    print("F1")
    print(answer)

0.9778869778869779
F1
0.6666666666666666
0.9721518987341772
F1
0.6666666666666667
0.9566326530612245
F1
0.5853658536585366
0.9817767653758542
F1
0.7142857142857143
0.9626168224299065
F1
0.5294117647058824


In [ ]:
import pandas as pd
import glob
import numpy as np
np.set_printoptions(precision=4,suppress=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
import nltk 
import spacy
from tqdm import tqdm
import re
import pickle

In [5]:

def preprocessing(text):
        try:
            text = text.lower()
            # clean-up url
            text = GRUBER_URLINTEXT_PAT.sub('', text)
            # remove time
            text = TIME_CLEAN.sub('', text)
            # clean-up numbers
            text = NUMBER_CLEAN.sub('', text)
            text = PUNCT_CLEAN.sub(r'\1\2',text)
            text = TWEET_CLEAN.sub('', text)
            text = HASH_CLEAN.sub(r'\1', text)
            text = DASH_CLEAN.sub(r'\1 ', text)
            text = SPACE_CLEAN.sub(r'\1', text)
            text = RT_CLEAN.sub(r'', text)
            return ' '.join([word for word in text.split(' ') if word not in stopwords]).strip()
        except:
            return ''

def testing_main(testFileName):

    frames = []
    #for f in glob.glob(testFileName):
    for f in glob.glob('test_data_1a_utf8_sample.csv'):
        frames.append(pd.read_csv(f))
    ##df = pd.read_csv('test_data_1a_utf8_sample.csv')
    #df = pd.read_csv(testFileName)
    raw_df = pd.concat(frames)
    
    df = pd.concat([raw_df[raw_df.Tag.isnull()], raw_df[raw_df.Tag=='Prio-Fall']])
    df = df[df.kind == 1]

    GRUBER_URLINTEXT_PAT = re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019]))')
    NUMBER_CLEAN = re.compile(r'\d+(:?[.,]?\d+)*')
    TIME_CLEAN = re.compile(r'\d+:\d+(?:\s?[u]hr)?')
    PUNCT_CLEAN = re.compile(r'(\s?)[\.\,()\[\]\{\}\^\:\'\"\/;…/%//“=–><&]+(\s?)')
    DASH_CLEAN = re.compile(r'([^et])[-]')
    TWEET_CLEAN = re.compile(r'@\w+')
    HASH_CLEAN = re.compile(r'#(\w+)')
    SPACE_CLEAN = re.compile(r'(\s)\s+')
    RT_CLEAN = re.compile(r'^rt\b')

    with open('./stopwords-de.txt') as f:
        stopwords = f.read().split('\n')
        
    documents = [] # Content
    docs = [] # Content through spacy
    labels = [] # SichterName
    tags = [] # Tag

    for (idx, row) in tqdm(df.iterrows()):
        labels.append(row.SichterName)
        documents.append(preprocessing(row.Content))
    #     docs.append([token for token in nlp(documents[-1])])
        tags.append(str(row.Tag))
    
    
    # Save processed rows
    pickle.dump(documents, open('documents.pl', 'wb'))
    pickle.dump(labels, open('labels.pl', 'wb'))
    pickle.dump(tags, open('tags.pl', 'wb'))

,Host,kind,Datetime,Content,Url,Authorid,entryid,Id,TwistId,TopicId,TopicName,IsInitialEntry,IsPrivate
0,www.facebook.com,1,02.06.2017 08:02:03,Ich hab da mal ein Video vom neuen Smart Route...,https://www.facebook.com/deutschetelekom/posts...,11249514,0000c6fe-b9b4-4598-bebc-c29c0d3e3c5a,41401286,ee88b4f5-09fc-4b10-ba97-ef29860d1914,1,Client 388,True,False
1,twitter.com,1,04.05.2017 15:23:29,"RT @Telekom_hilft : meddl Leude , servus und h...",https://twitter.com/xitzjustxxxx/status/860122...,10943057,0001bfde-70e0-425f-a78a-4efb92eedeef,39942526,edb537c8-a2de-462e-9c24-b7e7c5dc8252,551631,Alle,True,False
2,twitter.com,1,04.05.2017 15:42:29,@Telekom_hilft Wird WLAN eines Tages mal wiede...,https://twitter.com/tom_xcs/status/86012749461...,11050655,0005715b-7d3d-4ab2-af48-688ffacd4eb4,39943533,976fc474-3ee6-470b-893c-e72bd17b2d9f,551631,Alle,True,False
3,twitter.com,1,27.06.2017 09:15:02,@Telekom_hilft ist in 01234567890 irgendeine S...,https://twitter.com/tweetkiba/status/879598934...,266235,0006432e-4d65-4197-947c-b0e6a6ee67ae,42432037,d69f0a48-08bb-4d91-9d41-4bf798bf40d1,551631,Alle,True,False
4,twitter.com,1,17.05.2017 21:34:55,"Hey , ich bin für #SurfenImPark :) https://t.c...",https://twitter.com/USMC_AKK/status/8649272270...,11196925,00068d65-d3df-4623-820e-8d050c1e5f01,40613868,02da374c-465b-425f-bbe3-be628295e3b4,551631,Alle,True,False
